In [43]:
#import pandas and os
import pandas as pd
import os

## Merging 12 months of data in single file

In [44]:
# test if data url works
# df = pd.read_csv("Sales_Data/Sales_April_2019.csv")
# df.head()

In [45]:
files = [file for file in os.listdir('Sales_Data')]

all_months_data =  pd.DataFrame()

for file in files:
    if file != ".DS_Store":
#         print(file)
        df = pd.read_csv("Sales_Data/"+file)
        all_months_data = pd.concat([all_months_data, df])

In [46]:
all_months_data.to_csv("all_data.csv")

In [48]:
all_data = pd.read_csv("all_data.csv")
all_data.shape

(186850, 7)

## Data Cleanup

### Find all NaN

In [47]:
# row_has_NaN = all_months_data.isnull().any(axis=1)
# rows_with_NaN = all_months_data[row_has_NaN]
# rows_with_NaN.shape
# all_months_data = all_months_data.dropna(how='any')
# all_months_data.shape